In [ ]:
!pip install -U python-dotenv openai

In [ ]:
from dotenv import load_dotenv

load_dotenv(dotenv_path="/home/ubuntu/work/edu-src-all/.env")

In [ ]:
from openai import OpenAI
import os

client = OpenAI()

response = client.responses.create(
    model=os.getenv("OPENAI_DEFAULT_MODEL"),
    input="지도학습과 비지도 학습의 차이에 대해서 설명해줘 "
)

print(response.output_text)  


In [ ]:
response

In [ ]:
from openai import OpenAI

client = OpenAI()

response = client.responses.create(
    model="gpt-4.1-mini",
    input=[
        {
            "role": "user",
            "content": [
                { "type": "input_text", "text": "이 이미지에 뭐가 보이지?" },
                {
                    "type": "input_image",
                    "image_url": "https://images.ctfassets.net/kftzwdyauwt9/e632747f-9587-47a4-60779b6e0c90/cf75112eedea676e9deed512d191d1ac/planning-for-agi-and-beyond.jpg?w=1920&q=90&fm=webp"
                }
            ]
        }
    ]
)

print(response.output_text)  

In [ ]:
response

In [ ]:
import base64
import requests
from openai import OpenAI

client = OpenAI()

image_url = "https://i.namu.wiki/i/IdqeC7GAbMF20iGgFDwKqn5nOTgaQdKHa9ln_GhDjv54wKkaS7tcCJ5eciKgyUp17XzVQnscB1-FD12m6JkJp9BYmkSkvn0U30W9cJA0lsyqsiZVZedVfepgxDIU52WKPIvEHheLa9MyoKCZnvh7Qw.webp"

img_bytes = requests.get(image_url, timeout=10).content

b64 = base64.b64encode(img_bytes).decode("utf-8")
data_url = f"data:image/jpeg;base64,{b64}"

response = client.responses.create(
    model=os.getenv("OPENAI_DEFAULT_MODEL"),
    input=[
        {
            "role": "user",
            "content": [
                {"type": "input_text", "text": "이 이미지에 뭐가 보이지?"},
                {
                    "type": "input_image",
                    "image_url": data_url
                }
            ]
        }
    ]
)

print(response.output_text)


In [ ]:
from openai import OpenAI

client = OpenAI()

response = client.responses.create(
    model=os.getenv("OPENAI_DEFAULT_MODEL"),
    input=[
        {
            "role": "user",
            "content": [
                { "type": "input_text", "text": "이 파일의 내용은 뭐야?" },
                {
                    "type": "input_file",
                    "file_url": "https://www.berkshirehathaway.com/letters/2024ltr.pdf"
                }
            ]
        }
    ]
)

print(response.output_text)  

In [ ]:
response

In [ ]:
from openai import OpenAI

client = OpenAI()

response = client.responses.create(
    model=os.getenv("OPENAI_DEFAULT_MODEL"),
    tools=[{ "type": "web_search_preview" }],
    input="가장 최근에 월드컵에서 우승한 국가는 어디지? ",
)

print(response.output_text)  

In [ ]:
response

In [ ]:
from openai import OpenAI
from openai.types.responses import ResponseTextDeltaEvent

client = OpenAI()

response = client.responses.create(
    model=os.getenv("OPENAI_DEFAULT_MODEL"),
    instructions="당신은 친절한 어시스턴트입니다.",
    input="PLM과 LLM의 차이는?",
    stream=True
)

for event in response:
  if isinstance(event, ResponseTextDeltaEvent):
    print(event.delta, end='')


In [ ]:
import os
from openai import OpenAI

client = OpenAI()

tools = [
    {
        "type": "function",
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
          "type": "object",
          "properties": {
              "location": {
                  "type": "string",
                  "description": "The city and state, e.g. San Francisco, CA",
              },
              "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
          },
          "required": ["location", "unit"],
        }
    }
]

response = client.responses.create(
    model=os.getenv("OPENAI_DEFAULT_MODEL"),
    tools=tools,
    input="서울의 오늘 날씨는 어때?",
    tool_choice="auto"
)

print(response)

In [ ]:
from openai import OpenAI
client = OpenAI()

response = client.responses.create(
    model="o4-mini",
    input="Strawberry에는 R이 몇개 있지?",
    reasoning={
        "effort": "high"
    }
)

print(response)


In [ ]:
file = client.files.create(
    file=open("data/소나기.pdf", "rb"),
    purpose='assistants'
)
print(f"업로드된 파일 ID: {file.id}")

In [ ]:
vector_store = client.vector_stores.create(
    name="국어 교재"
)

In [ ]:
client.vector_stores.files.create(
    vector_store_id=vector_store.id,
    file_id=file.id
)
print(f"Vector Store ID: {vector_store.id}")

In [ ]:
response = client.responses.create(
    model="gpt-4o-mini",
    instructions="당신은 친절한 국어선생님입니다. 제공된 문서를 참고하여 질문에 답변하세요. 답을 모르겠으면 모르겠다고 답해야 하고 답을 지어내면 안됩니다.",
    input=[
        {
            "role": "user",
            "content": "소년과 소녀는 어디에서 처음 만났나?"
        }
    ],
    tools=[
        {
            "type": "file_search",
            "vector_store_ids": [vector_store.id],
            "max_num_results": 4
        }
    ]
)

print(response.output_text)  

In [ ]:
response2 = client.responses.create(
    model="gpt-4o-mini",
    instructions="당신은 친절한 국어선생님입니다. 제공된 문서를 참고하여 질문에 답변하세요.",
    input=[
        {
            "role": "user",
            "content": "소년과 소녀는 어디에서 처음 만났나?"
        },
        {
            "role": "assistant",
            "content": response.output_text
        },
        {
            "role": "user",
            "content": "그때 소녀는 무엇을 하고 있었나?"
        }
    ],
    tools=[
        {
            "type": "file_search",
            "vector_store_ids": [vector_store.id],
            "max_num_results": 4
        }
    ]
)

print(response2.output_text)  

In [ ]:
import os
from openai import OpenAI

client = OpenAI()

response = client.responses.create(
    model=os.getenv("OPENAI_DEFAULT_MODEL", "gpt-4o-mini"),
    tools=[{
        "type": "code_interpreter",
        "container": {"type": "auto"}  # 자동으로 컨테이너 생성/재사용
    }],
    input=[{
        "role": "user",
        "content": "3x + 11 = 14라는 방정식을 풀려고 하는데 도와줄 수 있어?"
    }]
)

print(response.output_text)

In [ ]:
response